In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import country_converter as coco
import rich
import plotly.express as px
import plotly.figure_factory as ff

In [3]:
def load_data():

    remote_ratio_map = {
        0: "No remote work",
        50: "Partially remote",
        100: "Fully remote",
    }

    experience_level_map = {
        "EN": "Junior",
        "MI": "Intermediate",
        "SE": "Expert",
        "EX": "Director",
    }

    employment_type_map = {
        "PT": "Part-time",
        "FT": "Full-time",
        "CT": "Contract",
        "FL": "Freelance",
    }

    company_size_map = {
        "S": "Small",
        "M": "Medium",
        "L": "Large",
    }

    data_path = r"..\data\salaries.csv"

    df: pd.DataFrame = (
        pd.read_csv(data_path)
        .astype(
            {
                "work_year": "category",
                "experience_level": pd.CategoricalDtype(
                    experience_level_map.keys(), ordered=True
                ),
                "employment_type": pd.CategoricalDtype(
                    employment_type_map.keys(), ordered=True
                ),
                "company_size": pd.CategoricalDtype(
                    company_size_map.keys(), ordered=True
                ),
                "remote_ratio": pd.CategoricalDtype(
                    remote_ratio_map.keys(), ordered=True
                ),
            }
        )
        .drop(columns=["salary", "salary_currency"])
    )

    df["working_overseas"] = df.employee_residence != df.company_location

    df = df.replace(
        {
            "remote_ratio": remote_ratio_map,
            "experience_level": experience_level_map,
            "employment_type": employment_type_map,
            "company_size": company_size_map,
            "employee_residence": dict(
                zip(
                    df.employee_residence.unique(),
                    coco.convert(df.employee_residence.unique(), to="name_short"),
                )
            ),
            "company_location": dict(
                zip(
                    df.company_location.unique(),
                    coco.convert(df.company_location.unique(), to="name_short"),
                )
            ),
        },
    )

    return df


df = load_data()

In [6]:

def chloropleth_empl_country(
    df: pd.DataFrame, work_years: list[int], experience_levels: list[str]
):

    cn = (
        df.query("work_year in @work_years and experience_level in @experience_levels")
        .employee_residence.value_counts()
        .to_frame()
        .reset_index()
        .rename(columns={"index": "country", "employee_residence": "no_of_empls"})
    )

    fig = px.choropleth(
        cn,
        locations=coco.convert(names=cn.country, to="ISO3"),
        color="no_of_empls",
        range_color=(0, cn.no_of_empls.quantile(0.98)),
        hover_name="country",
        labels={
            "no_of_empls": "No. of Employees",
        },
    )
    print(cn)
    print(fig)
    return fig

chloropleth_empl_country(df, work_years=[2020, 2021, 2022], experience_levels=['Junior', 'Director'])

               country  no_of_empls
0        United States           72
1                India           20
2              Germany           10
3               France            9
4       United Kingdom            8
5               Canada            5
6             Pakistan            4
7             Portugal            4
8            Australia            3
9              Denmark            3
10         Netherlands            3
11               Spain            3
12              Brazil            3
13             Belgium            2
14               Italy            2
15               Japan            2
16              Russia            2
17                Iraq            1
18          Luxembourg            1
19            Colombia            1
20               Kenya            1
21              Jersey            1
22             Nigeria            1
23             Ukraine            1
24             Tunisia            1
25             Algeria            1
26       Aland Islands      

In [3]:
df.sample(n=5, random_state=42)

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,working_overseas
533,2022,Intermediate,Full-time,Data Engineer,84711,Spain,Fully remote,Spain,Medium,False
139,2022,Expert,Full-time,Data Engineer,105000,United States,Fully remote,United States,Medium,False
88,2022,Expert,Full-time,Data Scientist,191475,United States,Fully remote,United States,Medium,False
841,2021,Intermediate,Full-time,Research Scientist,62649,France,Partially remote,France,Medium,False
985,2021,Expert,Full-time,Machine Learning Engineer,24342,India,Fully remote,India,Large,False


In [4]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
work_year,1095.0,3.0,2022.0,795.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
experience_level,1095,4,Expert,623,NaN,NaN,NaN,NaN,NaN,NaN,NaN
employment_type,1095,4,Full-time,1070,NaN,NaN,NaN,NaN,NaN,NaN,NaN
job_title,1095,59,Data Scientist,270,NaN,NaN,NaN,NaN,NaN,NaN,NaN
salary_in_usd,1095.0,NaN,NaN,NaN,121612.727854,67546.074808,2324.0,72750.0,117000.0,160000.0,600000.0
employee_residence,1095,64,United States,711,NaN,NaN,NaN,NaN,NaN,NaN,NaN
remote_ratio,1095,3,Fully remote,687,NaN,NaN,NaN,NaN,NaN,NaN,NaN
company_location,1095,59,United States,739,NaN,NaN,NaN,NaN,NaN,NaN,NaN
company_size,1095,3,Medium,691,NaN,NaN,NaN,NaN,NaN,NaN,NaN
working_overseas,1095,2,False,1030,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   work_year           1095 non-null   category
 1   experience_level    1095 non-null   category
 2   employment_type     1095 non-null   category
 3   job_title           1095 non-null   object  
 4   salary_in_usd       1095 non-null   int64   
 5   employee_residence  1095 non-null   object  
 6   remote_ratio        1095 non-null   category
 7   company_location    1095 non-null   object  
 8   company_size        1095 non-null   category
 9   working_overseas    1095 non-null   bool    
dtypes: bool(1), category(5), int64(1), object(3)
memory usage: 41.5+ KB
None


In [81]:
def pie_comp_size(df: pd.DataFrame, work_years: list[int]):

    comp_size = (
        df.query("work_year in @work_years")
        .company_size
        .value_counts()
        .sort_index()
    )
    print(comp_size)
    fig = px.pie(
        names=comp_size.index,
        values=comp_size.values,
        color=comp_size.index,
        # color_discrete_map={
        #     'Small': '#636efa',
        #     'Medium': '#ef553b',
        #     'Large': '#00cc96'
        # },
    )
    fig.update_traces(
        textinfo='label+percent+value',
    )

    return fig


work_years = [
    2020,
    2021,
    2022,
    ]
pie_comp_size(df, work_years)

Small     106
Medium    691
Large     298
Name: company_size, dtype: int64


In [24]:
px.box(
    df,
    x='experience_level',
    y='salary_in_usd',
    color='experience_level',
    labels={
        'experience_level': 'Experience Level',
        'salary_in_usd': 'Salary in USD'
    }
)

In [6]:
# Employees by Job Title
def bar_job_title(df, work_years: list[int]):

    if work_years == []:
        return None, None

    df_sub = df.query("work_year in @work_years")

    top_jt = df_sub.job_title.value_counts().head(20)

    jt = (
        df_sub.groupby(by=["job_title", "experience_level"])
        .salary_in_usd.count()
        .reset_index()
        .rename(columns={"salary_in_usd": "no_of_empls"})
        .set_index("job_title")
        .loc[top_jt.index.to_list()]
        .reset_index()
    )

    no_1_jt = jt.query("job_title == @top_jt.head(1).index.values[0]")

    fig = px.bar(
        jt,
        x="job_title",
        y="no_of_empls",
        color="experience_level",
        title=f'Top Job Titles in the years: {", ".join(str(y) for y in work_years if y is not None)}',
        labels={
            "job_title": "Job Title",
            "no_of_empls": "No. of Employees",
            "experience_level": "Experience Level",
        },
    )
    # fig.update_layout(
    #     height=600,
    #     width=500
    # )

    return no_1_jt, fig


no_1, fig = bar_job_title(df=df, work_years=[2020, 2021, 2022])
print(no_1)
print(f"Top Job Title: {no_1.job_title.iloc[0]}")
print(f"Total No.of Employees: {no_1.no_of_empls.sum()} ")
# top_10_jt = jt.groupby('job_title')[['no_of_empls']].sum().sort_values(by='no_of_empls', ascending=False)
# df_sub.query('job_title == @jt_1.index[0]')
fig

        job_title experience_level  no_of_empls
0  Data Scientist           Junior           31
1  Data Scientist     Intermediate           77
2  Data Scientist           Expert          162
3  Data Scientist         Director            0
Top Job Title: Data Scientist
Total No.of Employees: 270 


In [7]:
def bar_sal_job_title(df, work_years: list[int]):

    if work_years == []:
        return None, None

    df_sub = df.query("work_year in @work_years")

    ms =(
        df_sub
        .groupby(['job_title', 'experience_level'])
        .salary_in_usd
        .median()
        .reset_index()
        .rename(columns={"salary_in_usd": "median_salary"})
    )

    top_ms = ms.groupby('job_title').median_salary.sum().nlargest(20)

    no_1_ms = ms.query("job_title == @top_ms.head(1).index.values[0]")

    ms10 = (
        ms
        .set_index('job_title')
        .loc[top_ms.index.to_list()]
        .reset_index()

    )

    fig = px.bar(
        ms10,
        x="job_title",
        y="median_salary",
        color="experience_level",
    )

    return no_1_ms, fig

no_1, fig = bar_sal_job_title(df=df, work_years=[2020, 2021, 2022])
print(no_1)
print(f"Highest paying Job Title: {no_1.job_title.iloc[0]}")
print(f"Median Salary: {no_1.median_salary.sum()} ")
fig


# df.loc[df.salary_in_usd.nlargest(10).index.to_list()]
# ms.loc[ms.median_salary.nlargest(10).index.to_list()]
# ms.groupby('job_title').agg({'median_salary': 'sum'})

# top_ms
# no_1_ms
# ms10
# fig

                   job_title experience_level  median_salary
216  Principal Data Engineer           Junior            NaN
217  Principal Data Engineer     Intermediate            NaN
218  Principal Data Engineer           Expert       192500.0
219  Principal Data Engineer         Director       600000.0
Highest paying Job Title: Principal Data Engineer
Median Salary: 792500.0 


In [8]:
# No. of Employees by Country
def chloropleth_empl_country(
    df: pd.DataFrame, work_years: list[int], experience_levels: list[str]
):
    cn = (
        df.query("work_year in @work_years and experience_level in @experience_levels")
        .employee_residence.value_counts()
        .to_frame()
        .reset_index()
        .rename(columns={"index": "country", "employee_residence": "no_of_empls"})
    )
    fig = px.choropleth(
        cn,
        locations=coco.convert(names=cn.country, to="ISO3"),
        color="no_of_empls",
        range_color=(0, cn.no_of_empls.quantile(0.98)),
#         title=f"Distribution of Employees in years: \
# [{', '.join(str(y) for y in work_years)}] \
# and experience_levels: \
# [{', '.join(experience_levels)}]",
        hover_name="country",
        # animation_frame=df.work_year.unique(),
        labels={
            "no_of_empls": "No. of Employees",
        },
        projection="van der grinten"
    )
    return fig


# print(cn)
# print(cn.values)
# print(cn.no_of_empls.quantile(0.98))


experience_levels = [
    "Junior",
    "Intermediate",
    "Expert",
    "Director",
]
work_years = [
    2020,
    # 2021,
    2022,
]
chloropleth_empl_country(df, work_years=work_years, experience_levels=experience_levels)
# fig

In [9]:
def chloropleth_sal_country(
    df: pd.DataFrame, work_years: list[int], experience_levels: list[str]
):

    sal_by_country = (
        df.query("work_year in @work_years and experience_level in @experience_levels")
        .groupby(['salary_in_usd', 'company_location'])
        .size()
        .reset_index()
        .groupby('company_location')
        .median()
        .reset_index()
    )

    fig = px.choropleth(
        locations=coco.convert(names=sal_by_country.company_location, to="ISO3"),
        color=sal_by_country.salary_in_usd
    )

    return fig

experience_levels = [
    "Junior",
    "Intermediate",
    "Expert",
    "Director",
]
work_years = [
    2020,
    2021,
    2022,
]

chloropleth_sal_country(df, work_years=work_years, experience_levels=experience_levels)


In [10]:
def dist_sal_by_work_year(df):

    y2020 = df.query('work_year == 2020')
    y2021 = df.query('work_year == 2021')
    y2022 = df.query('work_year == 2022')

    hist_data = [y2020.salary_in_usd, y2021.salary_in_usd, y2022.salary_in_usd]
    group_labels = ['2020', '2021', '2022']

    fig = ff.create_distplot(
        hist_data,
        group_labels,
        show_hist=False,
    )
    fig.update_layout(
        title="Distribution of Salary by Work Year",
        font=dict(size=14)
    )
    return fig

dist_sal_by_work_year(df)

In [11]:
def dist_sal_by_exp_level(df: pd.DataFrame):
    exp_level_sal = df[['experience_level','salary_in_usd']]

    entry_salary = exp_level_sal.query("experience_level == 'Junior'")
    executive_salary = exp_level_sal.query("experience_level == 'Director'")
    mid_salary = exp_level_sal.query("experience_level == 'Intermediate'")
    senior_salary = exp_level_sal.query("experience_level == 'Expert'")

    hist_data = [
        entry_salary.salary_in_usd,
        mid_salary.salary_in_usd,
        senior_salary.salary_in_usd,
        executive_salary.salary_in_usd
        ]
    group_labels = ['Junior','Intermediate','Expert','Director']


    fig = ff.create_distplot(hist_data, group_labels, show_hist=False,)

    fig.update_layout(
        title="Distribution of Salary by Experience Level",
        font=dict(size=14)
    )
    return fig

dist_sal_by_exp_level(df)

In [12]:
def dist_sal_by_company_size(df):
    exp_level_sal = df[['experience_level','salary_in_usd']]

    c_size = df[['company_size','salary_in_usd']]
    small = exp_level_sal.loc[c_size['company_size']=='Small']
    medium = exp_level_sal.loc[c_size['company_size']=='Medium']
    large = exp_level_sal.loc[c_size['company_size']=='Large']

    hist_data = [small['salary_in_usd'],medium['salary_in_usd'],large['salary_in_usd']]
    group_labels = ['Small','Mid','Large']

    plt.figure(figsize=(20, 5))
    fig = ff.create_distplot(hist_data, group_labels, show_hist=False)

    return fig

dist_sal_by_company_size(df).show()

<Figure size 2000x500 with 0 Axes>

In [88]:
def dis_sal_by_empl_type(df: pd.DataFrame):
    # exp_level_sal = df[['experience_level','salary_in_usd']]

    # c_size = df[['company_size','salary_in_usd']]
    # small = exp_level_sal.loc[c_size['company_size']=='Small']
    # medium = exp_level_sal.loc[c_size['company_size']=='Medium']
    # large = exp_level_sal.loc[c_size['company_size']=='Large']

    empl_type_sal = df[['employment_type', 'salary_in_usd']]
    print(empl_type_sal.employment_type.unique())
    


# 'Part-time' < 'Full-time' < 'Contract' < 'Freelance'
    

    # hist_data = [small['salary_in_usd'],medium['salary_in_usd'],large['salary_in_usd']]
    # group_labels = ['Small','Mid','Large']

    # plt.figure(figsize=(20, 5))
    # fig = ff.create_distplot(hist_data, group_labels, show_hist=False)

    # return fig

dis_sal_by_empl_type(df)

['Full-time', 'Contract', 'Part-time', 'Freelance']
Categories (4, object): ['Part-time' < 'Full-time' < 'Contract' < 'Freelance']


In [13]:
experience_levels = [
    "Junior",
    "Intermediate",
    "Expert",
    "Director",
]
work_years = [
    2020,
    # 2021,
    2022,
]
cn = (
    df.query("work_year in @work_years and experience_level in @experience_levels")
    .groupby(by=["work_year", "employee_residence"])[
        ["employee_residence", "work_year"]
    ]
    .agg({"employee_residence": "count"})
    .rename(columns={"employee_residence": "no_of_empls"})
    .reset_index()
)
cn2 = (
    cn
    # .query("work_year in @work_years")
    .groupby("employee_residence")
    # .groups
    # .count()
    .agg(
        {
            "no_of_empls": "sum",
            #  'work_year': lambda x: x
        }
    )
    # .reset_index()
    # .rename(columns={'employee_residence': 'country'})
)

# fig = px.choropleth(
#     cn2,
#     locations=coco.convert(names=cn2.country, to='ISO3'),
#     color='no_of_empls',
#     range_color=(0, cn2.no_of_empls.quantile(0.98)),
#     title=f"Distribution of Employees in years: \
# [{', '.join(str(y) for y in work_years)}] \
# and experience_levels: \
# [{', '.join(experience_levels)}]",
#     hover_name='country',
# animation_frame=cn.work_year.unique(),
#     labels={
#         'no_of_empls': 'No. of Employees',
#     }
# )

# .value_counts()
# .agg(
#     {'employee_residence': 'count',
# 'work_year': lambda x: print(x)
# }
# )
# .to_frame()
# .reset_index()
# .rename(columns={'index': 'country', 'employee_residence': 'no_of_empls'})

cn2
# df.query("employee_residence == 'Aland Islands'")

,no_of_empls
employee_residence,
Aland Islands,1
Algeria,1
Argentina,3
Australia,6
Austria,4
Belgium,2
Bolivia,3
Brazil,4
Canada,24


In [14]:
# Overseas Workers
df.query("working_overseas == True")

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,working_overseas
33,2022,Junior,Full-time,Data Analyst,20000,Costa Rica,Partially remote,United States,Medium,True
77,2022,Junior,Full-time,Data Specialist,105000,Chile,Fully remote,United States,Large,True
119,2022,Junior,Full-time,AI Scientist,31766,Portugal,Fully remote,Spain,Medium,True
135,2022,Intermediate,Contract,NLP Engineer,60000,India,Fully remote,United States,Small,True
137,2022,Junior,Full-time,Business Data Analyst,50000,India,Fully remote,American Samoa,Large,True
...,...,...,...,...,...,...,...,...,...,...
1069,2021,Junior,Full-time,AI Scientist,18053,India,Fully remote,American Samoa,Small,True
1071,2020,Intermediate,Full-time,Data Scientist,62726,France,Partially remote,Luxembourg,Small,True
1075,2021,Junior,Part-time,AI Scientist,12000,Brazil,Fully remote,United States,Small,True
1083,2020,Intermediate,Full-time,Data Engineer,130800,Spain,Fully remote,United States,Medium,True


In [15]:
df.pivot_table(
    index="job_title",
    columns="experience_level",
    values="salary_in_usd",
    aggfunc="median",
)

experience_level,Junior,Intermediate,Expert,Director
job_title,,,,
3D Computer Vision Researcher,2324.0,5409.0,NaN,NaN
AI Scientist,18053.0,200000.0,55000.0,200000.0
Analytics Engineer,130000.0,85000.0,140000.0,162500.0
Applied Data Scientist,80018.5,105619.0,278500.0,NaN
Applied Machine Learning Scientist,30707.0,75000.0,92861.0,NaN
Applied Scientist,NaN,NaN,167137.5,NaN
BI Analyst,NaN,NaN,200000.0,NaN
BI Data Analyst,8715.0,52522.0,NaN,150000.0
Big Data Architect,NaN,NaN,99703.0,NaN


In [16]:
experience_levels = [
    "Junior",
    "Intermediate",
    "Expert",
    "Director",
]
work_years = [
    2020,
    # 2021,
    2022,
]
avg_sal = (
    df.query("work_year in @work_years and experience_level in @experience_levels")
    .groupby(['salary_in_usd', 'company_location'])
    .size()
    .reset_index()
    .groupby('company_location')
    .median()
    .reset_index()
)
avg_sal

,company_location,salary_in_usd,0
0,Albania,2324.0,1.0
1,Algeria,100000.0,1.0
2,American Samoa,50000.0,1.0
3,Argentina,50000.0,1.0
4,Australia,84848.0,1.0
5,Austria,68302.0,1.0
6,Belgium,68563.0,1.0
7,Bolivia,7500.0,1.0
8,Brazil,6306.0,1.0
9,Canada,90700.0,1.0
